In [ ]:
import torch
import numpy
import pandas
from discretize import generator, MarkovSampler

In [ ]:
# number of stages:
T = 3
# number of assets:
N = 3
# risk free interest rate:
rf = 0.0005
# additional dataset:
fee = 0.001
# size of MC:
size = 25
 
coeffs = pandas.read_csv("./data/coefficients.csv",index_col=0)
beta = torch.tensor(coeffs['beta'])

alpha = torch.tensor(coeffs['alpha'])
beta = torch.tensor(coeffs['beta'])
sigma = torch.tensor(coeffs['epsilon'])

In [ ]:
def f(alpha,sigma):
    def inner(random_state):
        return random_state.normal.Normal(alpha+1,sigma).sample()
    return inner

In [ ]:
# augmented Markovian process generator
def generator_augmented(random_state, size, T):
    # (r_it, r_Mt, epsilon_Mt, sigma^2_Mt)
    process = generator(random_state, size, T)
    market_return = process[:,:,0]
    process_aug = torch.cat((beta[:N]*(market_return[:,:,numpy.newaxis]-rf) + rf,process),axis=-1,)
    return process_aug

In [ ]:
# Markov chain discretization
sample_paths = generator(torch.distributions,size=1000, T=T)
return_sample_paths = sample_paths[:,:,0]
var_sample_paths = sample_paths[:,:,2]
price_sample_paths = torch.cumprod(torch.exp(return_sample_paths),axis=1)
markovian = MarkovSampler(generator,n_Markov_states=[1]+[100]*(T-1),n_sample_paths=size)
markovian.SA()
# augment to N+3 dimension
Markov_states = [None for _ in range(T)]
transition_matrix = markovian.transition_matrix
for t in range(T):
    market_return = markovian.Markov_states[t][:,0].reshape(-1,1)
    asset_return_market_exposure = beta[:N]*(market_return-rf) + rf
    Markov_states[t] = torch.cat((asset_return_market_exposure,markovian.Markov_states[t]), axis=1)

In [ ]:
import gurobipy
from statistics_ import rand_int,check_random_state
from exception import SampleSizeError,DistributionError
from measure import Expectation
import copy_ as deepcopy
from collections import abc
from numbers import Number
import time
import math

class StochasticModel(object):
    """The StochasticModel class"""
    def __init__(self, name=""):
        self._model = gurobipy.Model(env=gurobipy.Env(), name=name)
        # each and every instance must have state variables, local copy variables
        self.states = []
        self.local_copies = []
        # (discretized) uncertainties
        # stage-wise independent discrete uncertainties
        self.uncertainty_rhs = {}
        self.uncertainty_coef = {}
        self.uncertainty_obj = {}
        # indices of stage-dependent uncertainties
        self.uncertainty_rhs_dependent = {}
        self.uncertainty_coef_dependent = {}
        self.uncertainty_obj_dependent = {}
        # true uncertainties
        # stage-wise independent true continuous uncertainties
        self.uncertainty_rhs_continuous = {}
        self.uncertainty_coef_continuous = {}
        self.uncertainty_obj_continuous = {}
        self.uncertainty_mix_continuous = {}
        # stage-wise independent true discrete uncertainties
        self.uncertainty_rhs_discrete = {}
        self.uncertainty_coef_discrete = {}
        self.uncertainty_obj_discrete = {}
        # cutting planes approximation of recourse variable alpha
        self.alpha = None
        self.cuts = []
        # linking constraints
        self.link_constrs = []
        # number of discrete uncertainties
        self.n_samples = 1
        # number of state varibles
        self.n_states = 0
        # probability measure for discrete uncertainties
        self.probability = None
        # type of true problem: continuous/discrete
        self._type = None
        # flag to indicate discretization of true problem
        self._flag_discrete = 0
        # collection of all specified dim indices of Markovian uncertainties
        self.Markovian_dim_index = []
        # risk measure
        self.measure = Expectation

    def __getattr__(self, name):
        try:
            return getattr(self._model, name)
        except AttributeError:
            raise AttributeError("no attribute named {}".format(name))

    def addStateVars(
            self,
            *indices,
            lb=0.0,
            ub=1e+100,
            obj=0.0,
            vtype='C',
            name="",
            uncertainty=None,
            uncertainty_dependent=None
    ):
        # Add state variables in bulk. Generalize gurobipy.addVars() to
        # incorporate uncertainty in the objective function. Variables are added
        # as state variables and the corresponding local copy variables will be
        # added behind the scene

        state = self._model.addVars(
            *indices, lb=lb, ub=ub, obj=obj, vtype=vtype, name=name
        )
        local_copy = self._model.addVars(
            *indices, lb=lb, ub=ub, name=name + "_local_copy"
        )
        self._model.update()
        self.states += state.values()
        self.local_copies += local_copy.values()
        self.n_states += len(state)

        if uncertainty is not None:
            uncertainty = self._check_uncertainty(uncertainty, 0, len(state))
            if callable(uncertainty):
                self.uncertainty_obj_continuous[
                    tuple(state.values())
                ] = uncertainty
            else:
                self.uncertainty_obj[tuple(state.values())] = uncertainty


        if uncertainty_dependent is not None:
            uncertainty_dependent = self._check_uncertainty_dependent(
                uncertainty_dependent, 0, len(state)
            )
            self.uncertainty_obj_dependent[
                tuple(state.values())
            ] = uncertainty_dependent

        return state, local_copy
    
    def addVars(
            self,
            *indices,
            lb=0.0,
            ub=1e+100,
            obj=0.0,
            vtype='C',
            name="",
            uncertainty=None,
            uncertainty_dependent=None
    ):
        # Add variables in bulk. Generalize gurobipy.addVars() to
        # incorporate uncertainty in the objective function

        var = self._model.addVars(
            *indices, lb=lb, ub=ub, obj=obj, vtype=vtype, name=name
        )
        self._model.update()

        if uncertainty is not None:
            uncertainty = self._check_uncertainty(uncertainty, 0, len(var))
            if callable(uncertainty):
                self.uncertainty_obj_continuous[
                    tuple(var.values())
                ] = uncertainty
            else:
                self.uncertainty_obj[tuple(var.values())] = uncertainty

        if uncertainty_dependent is not None:
            uncertainty_dependent = self._check_uncertainty_dependent(
                uncertainty_dependent, 0, len(var)
            )
            self.uncertainty_obj_dependent[
                tuple(var.values())
            ] = uncertainty_dependent

        return var
    
    def addConstrs(
        self, generator, name="", uncertainty=None, uncertainty_dependent=None
    ):
        # to incorporate uncertainty on the RHS of the constraints.
        # If you want to add constraints with uncertainties on coefficients,
        # use addConstr() instead and add those constraints one by one
        constr = self._model.addConstrs(generator, name=name)
        self._model.update()

        if uncertainty is not None:
            uncertainty = self._check_uncertainty(uncertainty, 0, len(constr))
            if callable(uncertainty):
                self.uncertainty_rhs_continuous[
                    tuple(constr.values())
                ] = uncertainty
            else:
                self.uncertainty_rhs[tuple(constr.values())] = uncertainty

        if uncertainty_dependent is not None:
            uncertainty_dependent = self._check_uncertainty_dependent(
                uncertainty_dependent, 0, len(constr)
            )
            self.uncertainty_rhs_dependent[
                tuple(constr.values())
            ] = uncertainty_dependent

        return constr
    
    def addConstr(
            self,
            lhs,
            sense=None,
            rhs=None,
            name="",
            uncertainty=None,
            uncertainty_dependent=None,
    ):
        # Add a constraint to the model. Generalize gurobipy.addConstr()
        # to incorporate uncertainty in a constraint
        constr = self._model.addConstr(lhs, sense=sense, rhs=rhs, name=name)
        self._model.update()

        if uncertainty is not None:
            uncertainty = self._check_uncertainty(uncertainty, 1, 1)
            for key, value in uncertainty.items():
                # key can be a gurobipy.Var or "rhs"
                # Append constr to the key
                if type(key) == gurobipy.Var:
                    if callable(value):
                        self.uncertainty_coef_continuous[(constr, key)] = value
                    else:
                        self.uncertainty_coef[(constr, key)] = value
                elif type(key) == str and key.lower() == "rhs":
                    if callable(value):
                        self.uncertainty_rhs_continuous[constr] = value
                    else:
                        self.uncertainty_rhs[constr] = value
                else:
                    raise ValueError("wrong uncertainty key!")

        if uncertainty_dependent is not None:
            uncertainty_dependent = self._check_uncertainty_dependent(
                uncertainty_dependent, 1, 1
            )
            for key, value in uncertainty_dependent.items():
                # key can be a gurobipy.Var or "rhs"
                # Append constr to the key
                if type(key) == gurobipy.Var:
                    if not any(key is item for item in self._model.getVars()):
                        raise ValueError("wrong uncertainty key!")
                    self.uncertainty_coef_dependent[(constr, key)] = value
                elif type(key) == str and key.lower() == "rhs":
                    self.uncertainty_rhs_dependent[constr] = value
                else:
                    raise ValueError("wrong uncertainty key!")

        return constr
    
    def _check_uncertainty_dependent(
        self, uncertainty_dependent, flag_dict, list_dim
    ):
        # Make sure the input uncertainty location index is in the correct form.
        # Return a copied uncertainty to avoid making changes to mutable object
        # given by the users.
        if isinstance(uncertainty_dependent, abc.Mapping):
            if flag_dict == 0:
                raise TypeError("wrong uncertainty_dependent format!")
            for key, item in uncertainty_dependent.items():
                try:
                    item = int(item)
                    uncertainty_dependent[key] = item
                except (TypeError,ValueError):
                    raise ValueError("location index of individual component \
                                     of uncertainty_dependent must be integer!")
                self.Markovian_dim_index.append(item)

        elif isinstance(uncertainty_dependent, (abc.Sequence, torch.Tensor)):
            uncertainty_dependent = list(uncertainty_dependent)
            if len(uncertainty_dependent) != list_dim:
                raise ValueError(
                    "dimension of the scenario is {} while \
                    dimension of added object is {}!"
                    .format(len(uncertainty_dependent), list_dim)
                )
            self.Markovian_dim_index += uncertainty_dependent

        elif isinstance(uncertainty_dependent, Number):
            uncertainty_dependent = int(uncertainty_dependent)
            if list_dim != 1:
                raise ValueError(
                    "dimension of the scenario is 1 while \
                    dimension of added object is {}!"
                    .format(list_dim)
                )
            self.Markovian_dim_index.append(uncertainty_dependent)
        else:
            raise TypeError("wrong uncertainty_dependent format")
        return uncertainty_dependent
    
    def _check_uncertainty(self, uncertainty, flag_dict, list_dim):
        # Make sure the input uncertainty is in the correct form. Return a
        # copied uncertainty to avoid making changes to mutable object given by
        # the users.
        if isinstance(uncertainty, abc.Mapping):
            uncertainty = dict(uncertainty)
            for key, item in uncertainty.items():
                if callable(item):
                    if not self._type:
                        # add uncertainty for the first time
                        self._type = "continuous"
                    else:
                        # already added uncertainty
                        if self._type != "continuous":
                            raise SampleSizeError(
                                self._model.modelName,
                                self.n_samples,
                                uncertainty,
                                "infinite"
                            )
                    try:
                        item(torch.distributions)
                    except TypeError:
                        raise DistributionError(arg=False)
                    try:
                        float(item(torch.distributions))
                    except (ValueError,TypeError):
                        raise DistributionError(ret=False)
                else:
                    try:
                        item = torch.tensor(item, dtype=torch.float64)
                    except ValueError:
                        raise ValueError("Scenarios must only contains numbers!")
                    if item.ndim != 1:
                        raise ValueError(
                            "dimension of the distribution is {} while \
                            dimension of the added object is {}!"
                            .format(item.ndim, 1)
                        )
                    uncertainty[key] = list(item)

                    if not self._type:
                        # add uncertainty for the first time
                        self._type = "discrete"
                        self.n_samples = len(item)
                    else:
                        # already added uncertainty
                        if self._type != "discrete":
                            raise SampleSizeError(
                                self._model.modelName,
                                "infinite",
                                {key:item},
                                len(item)
                            )
                        if self.n_samples != len(item):
                            raise SampleSizeError(
                                self._model.modelName,
                                self.n_samples,
                                {key:item},
                                len(item)
                            )
            if flag_dict == 0:
                raise TypeError("wrong uncertainty format!")
        elif isinstance(uncertainty, abc.Callable):
            try:
                sample = uncertainty(torch.distributions)
            except TypeError:
                raise DistributionError(arg=False)
            if list_dim == 1:
                try:
                    float(sample)
                except (ValueError,TypeError):
                    raise DistributionError(ret=False)
            else:
                try:
                    sample = [float(item) for item in sample]
                except (ValueError,TypeError):
                    raise DistributionError(ret=False)
                if list_dim != len(uncertainty(torch.distributions)):
                    raise ValueError(
                        "dimension of the distribution is {} while \
                        dimension of the added object is {}!"
                        .format(len(uncertainty(torch.distributions)), list_dim)
                    )
            if not self._type:
                # add uncertainty for the first time
                self._type = "continuous"
            else:
                # already added uncertainty
                if self._type != "continuous":
                    raise SampleSizeError(
                        self._model.modelName,
                        self.n_samples,
                        uncertainty,
                        "infinite"
                    )
        elif isinstance(uncertainty, (abc.Sequence, torch.Tensor)):
            uncertainty = torch.tensor(uncertainty)
            if list_dim == 1:
                if uncertainty.ndim != 1:
                    raise ValueError("dimension of the scenarios is {} while \
                                     dimension of the added object is 1!"
                        .format(uncertainty.ndim)
                    )
                try:
                    uncertainty = [float(item) for item in uncertainty]
                except ValueError:
                    raise ValueError("Scenarios must only contains numbers!")
            else:
                # list to list
                if uncertainty.ndim != 2 or uncertainty.shape[1] != list_dim:
                    dim = None if uncertainty.ndim == 1 else uncertainty.shape[1]
                    raise ValueError("dimension of the scenarios is {} while \
                                     dimension of the added object is 1!"
                        .format(dim, uncertainty.ndim)
                    )
                try:
                    uncertainty = torch.tensor(uncertainty, dtype=torch.float64)
                except ValueError:
                    raise ValueError("Scenarios must only contains numbers!")
                uncertainty = [list(item) for item in uncertainty]
            if not self._type:
                self._type = "discrete"
                self.n_samples = len(uncertainty)
            else:
                if self._type != "discrete":
                    raise SampleSizeError(
                        self._model.modelName,
                        "infinite",
                        uncertainty,
                        len(uncertainty)
                    )
                if self.n_samples != len(uncertainty):
                    raise SampleSizeError(
                        self._model.modelName,
                        self.n_samples,
                        uncertainty,
                        len(uncertainty)
                    )
        else:
            raise TypeError("wrong uncertainty format!")

        return uncertainty
    
    def _discretize(self, n_samples, random_state, replace=True):
        # Discretize stage-wise independent continuous uncertainties.
        if hasattr(self,'_flag_discrete') and self._flag_discrete == 1: return
        # Discretize continuous true problem
        if self._type == "continuous":
            self.n_samples = n_samples
            # Order of discretization matters
            for key, dist in sorted(
                self.uncertainty_rhs_continuous.items(),
                key=lambda t: repr(t[0]),
            ):
                self.uncertainty_rhs[key] = [
                    dist(random_state) for _ in range(self.n_samples)
                ]
            for key, dist in sorted(
                self.uncertainty_obj_continuous.items(),
                key=lambda t: repr(t[0]),
            ):
                self.uncertainty_obj[key] = [
                    dist(random_state) for _ in range(self.n_samples)
                ]
            for key, dist in sorted(
                self.uncertainty_coef_continuous.items(),
                key=lambda t: repr(t[0]),
            ):
                self.uncertainty_coef[key] = [
                    dist(random_state) for _ in range(self.n_samples)
                ]
            for keys, dist in sorted(
                self.uncertainty_mix_continuous.items(),
                key=lambda t: repr(t[0]),
            ):
                for i in range(self.n_samples):
                    sample = dist(random_state)
                    for index, key in enumerate(keys):
                        if type(key) == gurobipy.Var:
                            if key not in self.uncertainty_obj.keys():
                                self.uncertainty_obj[key] = [sample[index]]
                            else:
                                self.uncertainty_obj[key].append(sample[index])
                        elif type(key) == gurobipy.Constr:
                            if key not in self.uncertainty_rhs.keys():
                                self.uncertainty_rhs[key] = [sample[index]]
                            else:
                                self.uncertainty_rhs[key].append(sample[index])
                        else:
                            if key not in self.uncertainty_coef.keys():
                                self.uncertainty_coef[key] = [sample[index]]
                            else:
                                self.uncertainty_coef[key].append(
                                    sample[index]
                                )
        # Discretize discrete true problem
        else:
            if n_samples > self.n_samples:
                raise Exception(
                    "n_samples should be smaller than the total number of samples!"
                )
            for key, samples in sorted(
                self.uncertainty_rhs.items(), key=lambda t: repr(t[0])
            ):
                self.uncertainty_rhs_discrete[key] = samples
                # numpy.random.choice does not work on multi-dimensional arrays
                drawed_indices = rand_int(
                    self.n_samples,
                    random_state,
                    size=n_samples,
                    probability=self.probability,
                    replace=replace,
                )
                self.uncertainty_rhs[key] = [
                    samples[index]
                    for index in drawed_indices
                ]
            for key, samples in sorted(
                self.uncertainty_obj.items(), key=lambda t: repr(t[0])
            ):
                self.uncertainty_obj_discrete[key] = samples
                drawed_indices = rand_int(
                    self.n_samples,
                    random_state,
                    size=n_samples,
                    probability=self.probability,
                    replace=replace,
                )
                self.uncertainty_obj[key] = [
                    samples[index]
                    for index in drawed_indices
                ]
            for key, samples in sorted(
                self.uncertainty_coef.items(), key=lambda t: repr(t[0])
            ):
                self.uncertainty_coef_discrete[key] = samples
                drawed_indices = rand_int(
                    self.n_samples,
                    random_state,
                    size=n_samples,
                    probability=self.probability,
                    replace=replace,
                )
                self.uncertainty_coef[key] = [
                    samples[index]
                    for index in drawed_indices
                ]
            self.n_samples_discrete = self.n_samples
            self.n_samples = n_samples
        self._flag_discrete = 1

In [ ]:
import gurobipy
from itertools import product
import numpy
import pandas
from statistics_ import check_random_state
from statistics_ import check_Markovian_uncertainty
from statistics_ import check_Markov_states_and_transition_matrix
from exception import MarkovianDimensionError
from collections import abc
import numbers
import math


class MSLP(object):
    # A multistage stochastic linear program composed of a sequence of StochasticModels.
    def __init__(
            self,
            size,
            T,
            bound=None,
            sense=1,
            outputFlag=0,
            discount=1.0,
            ctg=False,
            **kwargs):
        if (T < 2
                or discount > 1
                or discount < 0
                or sense not in [-1, 1]
                or outputFlag not in [0, 1]):
            raise Exception('Arguments of SDDP construction are not valid!')

        self.T = T
        self.size = size
        self.discount = discount
        self.bound = bound
        self.sense = sense
        self.n_Markov_states = 1
        self.dim_Markov_states = {}
        self.measure = 'risk neutral'
        self._type = 'stage-wise independent'
        self._individual_type = 'original'
        self._set_up_default_bound()
        self._set_up_model()
        self._set_up_model_attr(sense, outputFlag, kwargs)
        self._flag_discrete = 0
        self._flag_update = 0
        self.db = None
        self._flag_infinity = 0
        if ctg: self._set_up_CTG()

    def __repr__(self):
        sense = 'Minimization' if self.sense == 1 else 'Maximization'
        string = ("<SDDP instance {} {} {} problem, {} stages, "
            + "{} discount, {} known bound>")
        return string.format(sense, self.measure, self._type, self.T,
            self.discount, self.bound)

    def __getitem__(self, t):
        return self.models[t]

    def _set_up_default_bound(self):
        if self.bound is None:
            self.bound = -1000000000 if self.sense == 1 else 1000000000

    def _set_up_model(self):
        self.models = [StochasticModel(name=str(t)) for t in range(self.T)]

    def _set_up_model_attr(self, sense, outputFlag, kwargs):
        for t in range(self.T):
            m = self.models[t]
            m.Params.outputFlag = outputFlag
            m.setAttr('modelsense', sense)
            for k,v in kwargs.items():
                m.setParam(k,v)

    def add_Markovian_uncertainty(self, Markovian_uncertainty):
        # Add a Markovian continuous process.

        if hasattr(self, "Markovian_uncertainty") or hasattr(self,
        "Markov_states"):
            raise ValueError("Markovian uncertainty has already added!")
        self.dim_Markov_states=check_Markovian_uncertainty(Markovian_uncertainty
        ,self.size,self.T)
        self.Markovian_uncertainty = Markovian_uncertainty
        self._type = 'Markovian'

    def discretize(
            self,
            n_samples=None,
            random_state=None,
            replace=True,
            n_Markov_states=None,
            method='SA',
            n_sample_paths=None,
            Markov_states=None,
            transition_matrix=None,
            int_flag=0):
        # Discretize Markovian continuous uncertainty by k-means or (robust)
        # stochasitic approximation.

        if n_samples is not None:
            if isinstance(n_samples, (numbers.Integral, torch.int)):
                if n_samples < 1:
                    raise ValueError("n_samples should be bigger than zero!")
                n_samples = (
                    [1]
                    +[n_samples] * (self.T-1)
                )
            elif isinstance(n_samples, (abc.Sequence, torch.Tensor)):
                if len(n_samples) != self.T:
                    raise ValueError(
                        "n_samples list should be of length {} rather than {}!"
                        .format(self.T,len(n_samples))
                    )
                if n_samples[0] != 1:
                    raise ValueError(
                        "The first stage model should be deterministic!"
                    )
            else:
                raise ValueError("Invalid input of n_samples!")
            # discretize stage-wise independent continuous distribution
            random_state = check_random_state(random_state)
            for t in range(1,self.T):
                self.models[t]._discretize(n_samples[t],random_state,replace)
        if n_Markov_states is None and method != 'input': return
        if method == 'input' and (Markov_states is None or
            transition_matrix is None): return
        if n_Markov_states is not None:
            if isinstance(n_Markov_states, (numbers.Integral, torch.Tensor)):
                if n_Markov_states < 1:
                    raise ValueError("n_Markov_states should be bigger than zero!")
                n_Markov_states = (
                    [1]
                    +[n_Markov_states] * (self.T-1)
                )
            elif isinstance(n_Markov_states, (abc.Sequence, torch.Tensor)):
                if len(n_Markov_states) != self.T:
                    raise ValueError(
                        "n_Markov_states list should be of length {} rather than {}!"
                        .format(self.T,len(n_Markov_states))
                    )
                if n_Markov_states[0] != 1:
                    raise ValueError(
                        "The first stage model should be deterministic!"
                    )
            else:
                raise ValueError("Invalid input of n_Markov_states!")
        from discretize import MarkovSampler
        if method in ['RSA','SA','SAA']:
            markovian = MarkovSampler(
                f=self.Markovian_uncertainty,
                n_Markov_states=n_Markov_states,
                n_sample_paths=n_sample_paths,
                int_flag=int_flag,
            )
        if method in ['RSA','SA','SAA']:
            self.Markov_states,self.transition_matrix = getattr(markovian, method)()
        elif method == 'input':
            dim_Markov_states, n_Markov_states = (
                check_Markov_states_and_transition_matrix(
                    Markov_states=Markov_states,
                    transition_matrix=transition_matrix,
                    T=self.T,
                )
            )
            if dim_Markov_states != self.dim_Markov_states:
                raise ValueError("The dimension of the given sample path "
                    +"generator is not the same as the given Markov chain "
                    +"approximation!")
            self.Markov_states = Markov_states
            self.transition_matrix = [torch.tensor(item) for item in transition_matrix]
        self._flag_discrete = 1
        self.n_Markov_states = n_Markov_states
        if method in ['RSA','SA','SAA']:
            return markovian

In [ ]:
AssetMgt = MSLP(size=size, T=T, sense=-1, bound=200)
AssetMgt.add_Markovian_uncertainty(generator_augmented)

for t in range(T):
    m = AssetMgt[t]
    now, past = m.addStateVars(N+1, lb=0, obj=0, name='asset')
    if t == 0:
        buy = m.addVars(N, name='buy')
        sell = m.addVars(N, name='sell')
        m.addConstrs(now[j] == buy[j] - sell[j] for j in range(N))
        m.addConstr(
            now[N] == 100
            - (1+fee) * gurobipy.quicksum(buy[j] for j in range(N))
            + (1-fee) * gurobipy.quicksum(sell[j] for j in range(N))
        )
    elif t != T-1:
        sell = m.addVars(N, name='sell')
        buy = m.addVars(N, name='buy')
        capm = m.addVars(N, lb = -gurobipy.GRB.INFINITY, name='capm')
        idio = m.addVars(N, name='idio')
        m.addConstr(
            now[N] == (
                (1+rf) * past[N]
                - (1+fee) * gurobipy.quicksum(buy[j] for j in range(N))
                + (1-fee) * gurobipy.quicksum(sell[j] for j in range(N))
            )
        )
        m.addConstrs(
            now[j] == capm[j] + idio[j] + buy[j] - sell[j]
            for j in range(N)
        )
        for j in range(N):
            m.addConstr(past[j] == capm[j], uncertainty_dependent={past[j]:j})
            m.addConstr(past[j] == idio[j], uncertainty={past[j]:f(alpha[j],sigma[j])})
    else:
        v = m.addVar(obj=1, lb=-gurobipy.GRB.INFINITY, name='wealth')
        capm = m.addVars(N, lb = -gurobipy.GRB.INFINITY, name='capm')
        idio = m.addVars(N, name='idio')
        m.addConstr(v == gurobipy.quicksum(now[j] for j in range(N+1)))
        m.addConstrs(
            now[j] == capm[j] + idio[j]
            for j in range(N)
        )
        for j in range(N):
            m.addConstr(past[j] == capm[j], uncertainty_dependent={past[j]:j})
            m.addConstr(past[j] == idio[j], uncertainty={past[j]:f(alpha[j],sigma[j])})
        m.addConstr(now[N] == (1+rf) * past[N])

In [ ]:
AssetMgt.discretize(n_samples=100,method='input',Markov_states=Markov_states,transition_matrix=transition_matrix,random_state=888,)
# AssetMgt.set_AVaR(l=0.5, a=0.25)
# AssetMgt_SDDP = SDDP(AssetMgt)
# AssetMgt_SDDP.solve(max_iterations=10)

In [ ]:
# https://github.com/lingquant/msppy/blob/master/doc/source/examples/portfolio_optimization/portfolio.ipynb

# https://github.com/lingquant/msppy/blob/master/msppy/utils/statistics.py#L104
# https://github.com/lingquant/msppy/blob/master/msppy/sp.py#L163
# https://github.com/lingquant/msppy/blob/master/msppy/msp.py#L111

# https://optimization-online.org/wp-content/uploads/2019/05/7199.pdf